<a href="https://colab.research.google.com/github/skanelo/Face-Mask-Detection/blob/main/facemask_detector.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Mount Google Drive to Google Colab notebook**

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
#%cd gdrive/MyDrive/KITTI/darknet/
%cd gdrive/MyDrive/face_mask_detection

**Create the folder in which our yolo annotations will be saved**

In [ ]:
!mkdir mask_yolo_train

Run the py script that converts **.xlm** files into **.txt** files, which are compatible for yolo.

-   --input  = [path_of_folder_with_xml_files]
-   --output = [path_of_folder_with_txt_files]

In [ ]:
!python xml_to_yolo.py --input=annotations --output=mask_yolo_train

**Check:** Count how many items have been created (853 in our case)

In [ ]:
!ls mask_yolo_train// -1 | wc -l

Copy all the images from **images folder** into **mask_yolo_train folder**

In [ ]:
!cp -r images/. mask_yolo_train

At this point we have to split our data into **train and test** data.
I created one more folder: **mask_yolo_test** and I saved 86 of the images in this folder. Therest 767 images are going to be used for trainining

In [34]:
%cd mask_yolo_train
!ls -F | grep .png | wc -l
!ls -F | grep .txt | wc -l
%cd ..

%cd mask_yolo_test
!ls -F | grep .png | wc -l
!ls -F | grep .txt | wc -l
%cd ..

/content/gdrive/My Drive/face_mask_detection/mask_yolo_train
767
767
/content/gdrive/My Drive/face_mask_detection
/content/gdrive/My Drive/face_mask_detection/mask_yolo_test
86
86
/content/gdrive/My Drive/face_mask_detection


Create **train.txt** file and **test.txt** file which contain the absolute paths of each .txt annotation file of **mask_yolo_train** folder and **mask_yolo_test** folder, respectively

(These files are important for training and testing our yolo model)

In [ ]:
import os

for dataset, txtname in zip(['mask_yolo_train/', 'mask_yolo_test/'], ['train.txt', 'test.txt']):
    image_files = []
    # Given we are already being located into /face_mask_detection folder
    os.chdir(os.path.abspath(dataset))
    count=0
    for filename in os.listdir(os.getcwd()):
      if filename.endswith(".png"):
          image_files.append(os.path.abspath(os.getcwd()) + '/' + filename)

    os.chdir("..")
    with open(txtname, "w") as outfile:
        for image in image_files:
            outfile.write(image)
            outfile.write("\n")
        outfile.close()

In [ ]:
!cp train.txt ../darknet/data
!cp test.txt ../darknet/data

Change directory, go to the parent directory (MyDrive) and there clone darknet from AlexeyAB's repo

In [ ]:
%cd ..
!git clone https://github.com/AlexeyAB/darknet
%cd darknet

Cloning into 'darknet'...
remote: Enumerating objects: 3, done.
remote: Counting objects: 100% (3/3), done.
remote: Compressing objects: 100% (3/3), done.
remote: Total 14751 (delta 0), reused 3 (delta 0), pack-reused 14748
Receiving objects: 100% (14751/14751), 13.31 MiB | 6.48 MiB/s, done.
Resolving deltas: 100% (10031/10031), done.
Checking out files: 100% (2023/2023), done.
/content/gdrive/My Drive/darknet


Check the **GPU**

In [ ]:
!nvidia-smi

Modify Makefile in order to activate GPUs configuration

In [ ]:
# change makefile to have GPU, OPENCV and LIBSO enabled
!sed -i 's/OPENCV=0/OPENCV=1/' Makefile
!sed -i 's/GPU=0/GPU=1/' Makefile
!sed -i 's/CUDNN=0/CUDNN=1/' Makefile
!sed -i 's/LIBSO=0/LIBSO=1/' Makefile

!/usr/local/cuda/bin/nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2020 NVIDIA Corporation
Built on Wed_Jul_22_19:09:09_PDT_2020
Cuda compilation tools, release 11.0, V11.0.221
Build cuda_11.0_bu.TC445_37.28845127_0


Compile our source files

In [ ]:
!make

Get the weights of pretrained yolov3 model with 53 conv layers

In [ ]:
#!wget https://pjreddie.com/media/files/yolov3.weights
!wget https://pjreddie.com/media/files/darknet53.conv.74

--2021-04-08 21:13:24--  https://pjreddie.com/media/files/darknet53.conv.74
Resolving pjreddie.com (pjreddie.com)... 128.208.4.108
Connecting to pjreddie.com (pjreddie.com)|128.208.4.108|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 162482580 (155M) [application/octet-stream]
Saving to: ‘darknet53.conv.74’

darknet53.conv.74   100%[===================>] 154.96M  8.40MB/s    in 26s     

2021-04-08 21:13:51 (5.94 MB/s) - ‘darknet53.conv.74’ saved [162482580/162482580]



Change permissions

In [13]:
!chmod +x ./darknet

- Copy *train.txt* file from **face_mask_detection/** folder to **darknet/data/** folder

- Create *face_mask.names* file, which contains the categories of our detection task. ***Good*** if someone wears properly her/his mask) ***Bad*** if someone doesn't wear a mask or wears it incorrectly

- Create *face_mask.data* file

- Create *face_mask.cfg* file and modify it so as to meet our task's criteria. For full details: https://github.com/AlexeyAB/darknet

- Finally, **darknet/data/** folder should contain: *train.txt, face_mask.data, face_mask.names*, while **darknet/cfg/** should contain *face_mask.cfg*

In [ ]:
!pwd

/content/gdrive/My Drive/darknet


Execution command for training

In [12]:
!./darknet detector train data/face_mask.data cfg/face_mask.cfg backup/face_mask_last.weights -dont_show -i 0 -map -points 0

Streaming output truncated to the last 5000 lines.
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 82 Avg (IOU: 0.818633), count: 10, class_loss = 0.144345, iou_loss = 0.268339, total_loss = 0.412684 
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 94 Avg (IOU: 0.765994), count: 10, class_loss = 0.802218, iou_loss = 0.536030, total_loss = 1.338248 
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 106 Avg (IOU: 0.000000), count: 1, class_loss = 0.080269, iou_loss = 0.000000, total_loss = 0.080269 
 total_bbox = 247490, rewritten_bbox = 0.006869 % 
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 82 Avg (IOU: 0.801332), count: 12, class_loss = 0.811059, iou_loss = 0.429657, total_loss = 1.240716 
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 94 Avg (IOU: 0.843306), count: 4, class_loss = 0.518112, iou_loss = 0.150349, total_loss = 0.668461 
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 

**Check** the mAP0.5 of the best weights. **(87.16%)**

In [15]:
!./darknet detector map data/face_mask.data cfg/face_mask.cfg backup/face_mask_best.weights

 CUDA-version: 11000 (11020), cuDNN: 7.6.5, GPU count: 1  
 OpenCV version: 3.2.0
 0 : compute_capability = 600, cudnn_half = 0, GPU: Tesla P100-PCIE-16GB 
net.optimized_memory = 0 
mini_batch = 1, batch = 16, time_steps = 1, train = 0 
   layer   filters  size/strd(dil)      input                output
   0 Create CUDA-stream - 0 
 Create cudnn-handle 0 
conv     32       3 x 3/ 1    704 x 704 x   3 ->  704 x 704 x  32 0.856 BF
   1 conv     64       3 x 3/ 2    704 x 704 x  32 ->  352 x 352 x  64 4.568 BF
   2 conv     32       1 x 1/ 1    352 x 352 x  64 ->  352 x 352 x  32 0.508 BF
   3 conv     64       3 x 3/ 1    352 x 352 x  32 ->  352 x 352 x  64 4.568 BF
   4 Shortcut Layer: 1,  wt = 0, wn = 0, outputs: 352 x 352 x  64 0.008 BF
   5 conv    128       3 x 3/ 2    352 x 352 x  64 ->  176 x 176 x 128 4.568 BF
   6 conv     64       1 x 1/ 1    176 x 176 x 128 ->  176 x 176 x  64 0.508 BF
   7 conv    128       3 x 3/ 1    176 x 176 x  64 ->  176 x 176 x 128 4.568 BF
   8 Shortcu

Execution command for testing

In [21]:
!./darknet detector test data/face_mask.data cfg/face_mask.cfg backup/face_mask_best.weights

 CUDA-version: 11000 (11020), cuDNN: 7.6.5, GPU count: 1  
 OpenCV version: 3.2.0
 0 : compute_capability = 600, cudnn_half = 0, GPU: Tesla P100-PCIE-16GB 
net.optimized_memory = 0 
mini_batch = 1, batch = 16, time_steps = 1, train = 0 
   layer   filters  size/strd(dil)      input                output
   0 Create CUDA-stream - 0 
 Create cudnn-handle 0 
conv     32       3 x 3/ 1    704 x 704 x   3 ->  704 x 704 x  32 0.856 BF
   1 conv     64       3 x 3/ 2    704 x 704 x  32 ->  352 x 352 x  64 4.568 BF
   2 conv     32       1 x 1/ 1    352 x 352 x  64 ->  352 x 352 x  32 0.508 BF
   3 conv     64       3 x 3/ 1    352 x 352 x  32 ->  352 x 352 x  64 4.568 BF
   4 Shortcut Layer: 1,  wt = 0, wn = 0, outputs: 352 x 352 x  64 0.008 BF
   5 conv    128       3 x 3/ 2    352 x 352 x  64 ->  176 x 176 x 128 4.568 BF
   6 conv     64       1 x 1/ 1    176 x 176 x 128 ->  176 x 176 x  64 0.508 BF
   7 conv    128       3 x 3/ 1    176 x 176 x  64 ->  176 x 176 x 128 4.568 BF
   8 Shortcu

Auxiliary function for displaying on Colab our prediction.jpg image

In [17]:
# define helper functions
def imShow(path):
  import cv2
  import matplotlib.pyplot as plt
  %matplotlib inline

  image = cv2.imread(path)
  height, width = image.shape[:2]
  resized_image = cv2.resize(image,(3*width, 3*height), interpolation = cv2.INTER_CUBIC)

  fig = plt.gcf()
  fig.set_size_inches(18, 10)
  plt.axis("off")
  plt.imshow(cv2.cvtColor(resized_image, cv2.COLOR_BGR2RGB))
  plt.show()

imShow('predictions.jpg')

**Code to upload our video**

In [28]:
from google.colab import files
uploaded = files.upload()

Saving vid.mp4 to vid (1).mp4


**Use our detector on a video**
It saves the output in a .avi file. In order to watch it you have to download it.

In [30]:
!./darknet detector demo data/face_mask.data cfg/face_mask.cfg backup/face_mask_best.weights -dont_show vid1.mp4 -i 0 -out_filename res1.avi

 CUDA-version: 11000 (11020), cuDNN: 7.6.5, GPU count: 1  
 OpenCV version: 3.2.0
Demo
 0 : compute_capability = 600, cudnn_half = 0, GPU: Tesla P100-PCIE-16GB 
net.optimized_memory = 0 
mini_batch = 1, batch = 16, time_steps = 1, train = 0 
   layer   filters  size/strd(dil)      input                output
   0 Create CUDA-stream - 0 
 Create cudnn-handle 0 
conv     32       3 x 3/ 1    704 x 704 x   3 ->  704 x 704 x  32 0.856 BF
   1 conv     64       3 x 3/ 2    704 x 704 x  32 ->  352 x 352 x  64 4.568 BF
   2 conv     32       1 x 1/ 1    352 x 352 x  64 ->  352 x 352 x  32 0.508 BF
   3 conv     64       3 x 3/ 1    352 x 352 x  32 ->  352 x 352 x  64 4.568 BF
   4 Shortcut Layer: 1,  wt = 0, wn = 0, outputs: 352 x 352 x  64 0.008 BF
   5 conv    128       3 x 3/ 2    352 x 352 x  64 ->  176 x 176 x 128 4.568 BF
   6 conv     64       1 x 1/ 1    176 x 176 x 128 ->  176 x 176 x  64 0.508 BF
   7 conv    128       3 x 3/ 1    176 x 176 x  64 ->  176 x 176 x 128 4.568 BF
   8 Sh